In [1]:
import sys, os

#os.environ['nnUNet_raw_data_base'] = "/local/scratch/aranem/Lifelong-nnUNet-storage/nnUNet_raw"
#os.environ['nnUNet_preprocessed'] = "/local/scratch/aranem/Lifelong-nnUNet-storage/nnUNet_preprocessed"
#os.environ['RESULTS_FOLDER'] = "/local/scratch/aranem/Lifelong-nnUNet-storage/nnUNet_trained_models"
#os.environ['EVALUATION_FOLDER'] = "/local/scratch/aranem/Lifelong-nnUNet-storage/nnUNet_models_evaluation"

from nnunet_ext.run.run_inference import run_inference
from nnunet_ext.run.run_evaluation import run_evaluation



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

PARAM_SEARCH_FOLDER is not defined and nnU-Net extension cannot be used for a parameter search. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.



In [2]:
arch = '3d_fullres'
fold = '0'
device = '4'

trainer = 'nnUNetTrainerLWF'

# Prostate
trained_on = [79, 78, 77, 76]
use_model = [79, 78, 77, 76]
evaluate_on = 79
use_heads = [76, 77] #[79, 78, 77, 76]

# Hyppocampus
#trained_on = [99, 98, 97]
#use_model = [99, 98, 97]
#evaluate_on = 97
#use_head = 98  # 97

In [3]:
import os

# Adapt checkpoint
from nnunet_ext.scripts.update_checkpoints import modify_checkpoints
# This is always so, repeat for the new_checkpoints iterating over env variables
current_checkpoint_root = '/local/scratch/aranem/Lifelong-nnUNet-storage/'
old_vars = [os.path.join(current_checkpoint_root, var_name) for var_name in ['nnUNet_raw_data_base', 'nnUNet_preprocessed', 'nnUNet_trained_models', 'nnUNet_models_evaluation']]
new_vars = [os.environ['nnUNet_raw_data_base'], os.environ['nnUNet_preprocessed'], os.environ['RESULTS_FOLDER'], os.environ['EVALUATION_FOLDER']]
for current_checkpoint, new_checkpoint in zip(old_vars, new_vars):
    args = [sys.argv[0], arch, trainer, '-trained_on']
    args += [str(x) for x in trained_on]
    args += ['-f', fold, '-r', '-rw', current_checkpoint, new_checkpoint, '-use']
    args += [str(x) for x in use_model]
    sys.argv = args
    modify_checkpoints()

This will modify all checkpoints in the directory: /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_trained_models/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst.
This will modify all checkpoints in the directory: /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_trained_models/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst.
This will modify all checkpoints in the directory: /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_trained_models/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst.
This will modify all checkpoints in the directory: /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_trained_models/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst.


In [ ]:
# Preprocess relevant datasets
import nnunet.experiment_planning.nnUNet_plan_and_preprocess as exp_pp
sys.argv = [sys.argv[0], '-t', str(evaluate_on)]
exp_pp.main()
sys.argv = [sys.argv[0], '-t', str(trained_on[0])]
exp_pp.main()

In [3]:
for head in use_heads:
    # Extract predictions
    args = [sys.argv[0], arch, trainer, '-trained_on']
    args += [str(x) for x in trained_on]
    args += ['-f', fold, '-use_model']
    args += [str(x) for x in use_model]
    args += ['-evaluate_on', str(evaluate_on), '-d', device, '-use_head', str(head)]
    # print(args)
    sys.argv = args
    run_inference()

This model expects 1 input modalities for each image
Found 13 unique case ids, here are some examples: ['Case28' 'Case29' 'Case01' 'Case29' 'Case01' 'Case01' 'Case30' 'Case29'
 'Case33' 'Case28']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 13
number of cases that still need to be predicted: 13
emptying cuda cache
loading parameters for folds, [0]
2022-01-28 19:04:39.094391: Using dummy2d data augmentation
Updating the Loss based on the provided previous trainer
2022-01-28 19:04:39.538711: loading checkpoint /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_trained_models/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst/nnUNetTrainerLWF__nnUNetPlansv2.1/Generic_UNet/SEQ/fold_0/model_final_checkpoint.model train= False
2022-01-28 19:04:39.779149: The nnUNetTrainerLWF model trained on Task079_UCL, Tas

/gris/gris-f/guests/cgonzale/anaconda3/envs/cl/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


preprocessing /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_models_evaluation/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst/nnUNetTrainerLWF__nnUNetPlansv2.1/Generic_UNet/SEQ/head_Task076_DecathProst/fold_0/Preds_Task079_UCL/Case01.nii.gz
using preprocessor GenericPreprocessor
before crop: (1, 29, 384, 384) after crop: (1, 29, 384, 384) spacing: [3.00026011 0.46875599 0.46875066] 

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
before: {'spacing': array([3.00026011, 0.46875599, 0.46875066]), 'spacing_transposed': array([3.00026011, 0.46875599, 0.46875066]), 'data.shape (data is transposed)': (1, 29, 384, 384)} 
after:  {'spacing': array([1., 1., 1.]), 'data.shape (data is resampled)': (1, 87, 180, 180)} 

(1, 87, 180, 180)
preprocessing /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_models_evalu

/gris/gris-f/guests/cgonzale/anaconda3/envs/cl/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
before: {'spacing': array([3.29999995, 0.52083331, 0.52083331]), 'spacing_transposed': array([3.29999995, 0.52083331, 0.52083331]), 'data.shape (data is transposed)': (1, 23, 383, 383)} 
after:  {'spacing': array([1., 1., 1.]), 'data.shape (data is resampled)': (1, 76, 199, 199)} 

(1, 76, 199, 199)
preprocessing /gris/gris-f/guests/cgonzale/Code_Storage/Lifelong-nnUNet_storage/nnUNet_models_evaluation/nnUNet_ext/3d_fullres/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst/Task079_UCL_Task078_I2CVB_Task077_ISBI_Task076_DecathProst/nnUNetTrainerLWF__nnUNetPlansv2.1/Generic_UNet/SEQ/head_Task076_DecathProst/fold_0/Preds_Task079_UCL/Case27.nii.gz
using preprocessor GenericPreprocessor
before crop: (1, 23, 384, 384) after crop: (1, 23, 383, 384) spacing: [3.29999995 0.52083331 0.52083331] 

separate z, order in z is 0 order inplane is 3
separate z, order in z is 0 order inplane is 1
befor